In [1]:
from mido import MidiFile, MidiTrack, MetaMessage

In [55]:
input_file = '../songs/midi_songs/for those about to rock.mid'
output_file = '../songs/midi_songs/test-split.mid'
in_mid = MidiFile(input_file)
out_mid = MidiFile(ticks_per_beat=in_mid.ticks_per_beat)

In [56]:
drum_track = MidiTrack()
out_mid.tracks.append(drum_track)

# Collect all messages with their absolute times
all_messages = []
current_time = 0

# Iterate over tracks in the original MIDI file
for track in in_mid.tracks:
    current_time = 0
    for msg in track:
        current_time += msg.time
        if (isinstance(msg, MetaMessage) and msg.type in ('set_tempo', 'time_signature')) or \
           (msg.type in ('note_on', 'note_off') and msg.channel == 9) or \
           (msg.type not in ('note_on', 'note_off')):
            all_messages.append((current_time, msg))

# Sort messages by absolute time
all_messages.sort(key=lambda x: x[0])

# Convert back to delta times
last_time = 0
for i, (abs_time, msg) in enumerate(all_messages):
    delta = abs_time - last_time
    new_msg = msg.copy(time=delta)
    drum_track.append(new_msg)
    last_time = abs_time

# Save the new MIDI file with only drum notes
out_mid.save(output_file)